# RAG over a Large Codebase

## SciPy

**SciPy** is a fundamental open-source library for scientific computing in Python. It builds on top of **NumPy** and provides efficient implementations of many core algorithms for mathematics, science, and engineering. SciPy is widely used in academia and industry for numerical analysis, optimization, signal processing, and more.

### Key Features

* **Linear Algebra & Optimization:** Robust solvers for systems of equations, eigenvalue problems, and constrained/unconstrained optimization.  
* **Integration & Differential Equations:** Tools for numerical integration, ODE solvers, and quadrature.  
* **Signal & Image Processing:** Filtering, Fourier transforms, and image manipulation utilities.  
* **Statistics & Probability:** Random variables, hypothesis testing, and statistical distributions.  
* **Sparse Matrices:** Efficient storage and computation with large, sparse systems.  

SciPy underlies much of the Python scientific ecosystem, serving as the backbone for applications in physics, biology, engineering, data science, and machine learning.

### Resources

* 📖 [Documentation](https://docs.scipy.org/doc/scipy/)  
* 💻 [GitHub Repository](https://github.com/scipy/scipy)

---

## Goal

We will build a Retrieval-Augmented Generation (RAG) system to answer natural language questions about a codebase (~10,000 scripts). By combining a language model with a retriever over the SciPy documentation and source code, the system will help a new coder explore and understand complex source code, algorithms, and APIs.

#

## Part 1 – Setup and Preprocessing

In this part, we will complete the following.

1. **Documentation Preparation**

   * Obtain the provided project documentation (manuals, reference guides, tutorials, etc.).
   * Each section will be treated as text input for retrieval.
   * Ignore large binary files, images, and other non-text resources.

2. **Chunking**

   * Split the documentation into overlapping chunks:

     * Suggested: 300–500 tokens per chunk.
     * Overlap: 50–100 tokens to preserve context across boundaries.
   * Store metadata for each chunk, such as document name and section or page reference.

3. **Embedding Generation**

   * Use a sentence-transformer (e.g., `msmarco-distilbert-base-v3`) to generate embeddings for each chunk of documentation.
   * Save embeddings and metadata to disk for reuse.

4. **Vector Database**

   * Load embeddings into FAISS or Chroma.
   * Confirm you can perform a similarity search for a sampmake the flow of the lab smoother.


### Part 1 Step 1: Expanding Code Comments

Take a look at each of these helper scripts. To demonstrate you fully understand the code, you will expand the comments.   The comments should explain not only *what* each line of code does, but also *why* it is there and *how* it contributes to the larger goal of the pipeline.

You are welcome to use ChatGPT (or other tools) to help generate initial drafts of comments. However, you are ultimately responsible for understanding the details — you will be expected to **regurgitate this level of explanation on an exam without assistance**.



In [2]:
!pip install faiss-cpu

In [ ]:
def setup():
  from google.colab import drive
  drive.mount('/content/drive')
  %cd '/content/drive/MyDrive/'
  !pip install -q faiss-cpu transformers sentence-transformers
  !mkdir -p docs code

setup()

In [37]:
import os
import json
import faiss
import numpy as np
from pathlib import Path
from typing import List, Dict

import torch
from transformers import AutoTokenizer, AutoModel

from sentence_transformers import SentenceTransformer
models = {}
#models['code'] = SentenceTransformer("sentence-transformers/msmarco-distilbert-base-v3")
models['code'] = SentenceTransformer("BAAI/bge-m3")

models['docs'] = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# -------- 1. File Loader --------
def load_code_files(base_dir, exts={".cpp", ".h", ".c", ".f90", ".py"}):
    """
    What does it do? (1 sentence)
      It goes through a folder and all its subfolders to find files with certain extensions, then gives us their locations and contents.

    What are the inputs?
      base_dir (str): The main folder where the search starts.
      exts (set, optional): A group of file types to look for. If not provided, it defaults to the one set in the argument.

    What are the outputs?
      It gives us pairs of (file_path, file_content) for every file that matches the extensions.


    This function kicks off the process by collecting all relevant code or document files from the given folder.
    It supplies the text data needed for the next steps to work on.
    """
    for root, _, files in os.walk(base_dir):
        # Iterate over each file name found in the current directory.
        for f in files:
            # We check if the file extension is in the set of desired extensions.
            if os.path.splitext(f)[-1] in exts:
                try:
                    path = Path(root) / f
                    with open(path, "r", errors="ignore") as fh:
                        # Yield the string representation of the path and the entire content of the file.
                        yield str(path), fh.read()
                except Exception as e:
                    print(f"Skipping {f}: {e}")


# -------- 2. Chunker --------
def chunk_text(text: str, chunk_size=1000, overlap=0):
    """
    What does it do? (1 sentence)
      it splits a given text into smaller, overlapping chunks based on word count.

    What are the inputs?
      text (str): The input text to be chunked.
      chunk_size (int, optional): The maximum number of words in each chunk. Defaults to 1000.
      overlap (int, optional): The number of words to overlap between consecutive chunks. Defaults to 0.

    What are the outputs?
      A list of dictionaries, where each dictionary represents a chunk and contains the chunked text, its start word index, and its end word index.

    Large text documents or code files are too long to be processed directly by embedding models or language models.
    This function breaks them down into manageable pieces, preserving context between chunks through overlap, which is crucial for effective retrieval in the RAG system.
    """
    # Split the input text into a list of words.
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        piece = " ".join(words[i:i+chunk_size])
        # Check if the resulting chunk is not just empty space.
        if piece.strip():
            chunks.append({
                "text": piece,
                "start_word": i,
                "end_word": i+chunk_size
            })
    return chunks


# -------- 3. Embedding with CodeBERT --------
def embed_texts(model, texts):
    """
    What does it do? (1 sentence)
      Generates numerical vector embeddings for a list of text strings using a specified sentence transformer model.

    What are the inputs?
      model: The sentence transformer model to use for embedding.
      texts (List[str]): A list of text strings to embed.

    What are the outputs?
      A numpy array of embeddings, where each row corresponds to an input text string.
      It normalizes the embeddings to unit vectors, which is beneficial for cosine similarity calculations.

    This function converts the text chunks into a format that can be used for similarity search in the vector database.
    The embeddings capture the semantic meaning of the text, allowing the retriever to find chunks relevant to a user's query, even if the exact keywords are not present.
    """
    # Use the provided sentence transformer model to encode the list of texts.
    # batch_size: Process texts in batches to improve efficiency.
    # show_progress_bar: Display a progress bar during the encoding process.
    # convert_to_numpy: Convert the output embeddings to a numpy array.
    # normalize_embeddings: Normalize the embeddings to unit vectors, which is beneficial for cosine similarity calculations.
    embs = model.encode(
        texts,
        batch_size=16,
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    return embs



## FAISS

The next two code blocks introduce **FAISS** (Facebook AI Similarity Search), a library built for efficient similarity search across large collections of vectors. After we embed documentation chunks into high-dimensional vectors, FAISS gives us the data structures and algorithms needed to store them and quickly retrieve the closest matches to a query.

A key feature of FAISS is its support for different types of **indexes** for vector search. In this lab, we will start with the simplest option: **IndexFlatL2**.

* This index stores all vectors directly in memory.
* It computes exact Euclidean ($L^2$) distances for each query.
* It is best suited for datasets ranging from a few thousand up to a few hundred thousanke HNSW or IVF?


In [4]:
# -------- 4. Build Vector DB --------
def build_faiss_index(chunks_with_meta, dim):
    """
    Creates a FAISS index for efficient similarity search.
    """
    # Initialize a Flat L2 index. This index stores all vectors directly and uses Euclidean distance.
    index = faiss.IndexFlatL2(dim)
    # Extract embeddings from chunks, stack them into a numpy array, and ensure float32 type.
    embeddings = np.vstack([c["embedding"] for c in chunks_with_meta]).astype("float32")
    # Add the embeddings to the FAISS index.
    index.add(embeddings)
    return index



# -------- Example Query --------
def query_index(query, topk=5, code_or_doc = 'code'):
    """
    Searches the FAISS index for the most similar text chunks to a query.
    """

    # Load the appropriate FAISS index file from disk based on the 'code_or_doc' parameter.
    index = faiss.read_index("%s/code_chunks.index" % code_or_doc)

    # Embed the user's query using the correct model (code or docs) and cast to float32.
    q_emb = embed_texts(models[code_or_doc],[query]).astype("float32")

    # Perform the similarity search on the index with the query embedding and desired number of results (topk).
    # Returns distances and indices of the top matches.
    distances, indices = index.search(q_emb, topk)

    # Return the indices of the topk results (the first row of the indices array).
    return indices[0]

In [5]:

# -------- MAIN PIPELINE --------
def process_codebase(base_dir="scipy-main", code_or_doc = 'code'):
    all_chunks = []
    if code_or_doc == 'docs':
        exts = {'.rst'}
        chunk_size = 1000
    else:
        exts = {'.py'}
        chunk_size = 250
    for fpath, text in load_code_files(base_dir, exts):
        for chunk in chunk_text(text,chunk_size=chunk_size):
            chunk["file"] = fpath
            all_chunks.append(chunk)

    # Embed with CodeBERT
    texts = [c["text"] for c in all_chunks]

    embs = embed_texts(models[code_or_doc], texts)
    print('done with embed')
    for c, e in zip(all_chunks, embs):
        c["embedding"] = e

    # Save metadata
    with open("%s/chunks_metadata.json" % code_or_doc, "w") as out:
        for c in all_chunks:
            meta = {k: v for k, v in c.items() if k != "embedding"}
            out.write(json.dumps(meta) + "\n")

    # Build index

    dim = len(all_chunks[0]["embedding"])
    index = build_faiss_index(all_chunks, dim)
    faiss.write_index(index, "%s/code_chunks.index" % code_or_doc)
    print(f"Processed {len(all_chunks)} chunks from {base_dir}.")


#process_codebase("scipy-main/doc/source",code_or_doc = 'docs')   # run once for docs

In [ ]:
process_codebase("/content/drive/MyDrive/scipy-main/doc/source",code_or_doc = 'docs')

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

done with embed
Processed 641 chunks from /content/drive/MyDrive/DSF/Lab 2/scipy-main/doc/source.


In [ ]:

process_codebase("/content/drive/MyDrive/scipy-main",code_or_doc = 'code')


## Part 2: Testing

Now that you have built a vector-based retrieval tool on the **SciPy documentation**, the next step is to **qualitatively assess how well it works**. You will not compute numerical accuracy yet. Instead, you will explore retrieval behavior and record your judgments about when the system feels useful, misleading, or irrelevant.

You will use two supports for this process:

1. The **official SciPy documentation**
2. A **free ChatGPT interface** to act as an external “relevance judge.”

---

### 1. Design a Variety of Queries

Create at least **5 queries** covering different types of information you might want from SciPy. Use the templates below and adapt them to optimization, statistics, linear algebra, and sparse matrices:

* **Conceptual (high-level):**
  Ask about an algorithm, method, or mathematical concept.
  *Example:* `"trust-region methods in optimization"` or `"sparse matrix factorization"`.

* **Function/API-specific:**
  Target a known function, class, or module in SciPy.
  *Example:* `"scipy.optimize.minimize"` or `"scipy.sparse.linalg.cg"`.

* **Keyword-only:**
  Use just a technical term.
  *Example:* `"Poisson distribution"` or `"LU decomposition"`.

* **Natural language (descriptive):**
  Ask a full question in plain English.
  *Example:* `"How do I compute confidence intervals for a normal distribution in SciPy?"`

* **Edge cases:**
  Include queries that are vague, misspelled, or off-topic.
  *Example:* `"optimizashun"`, `"solver"`, `"deep learning"`.

---

### 2. Compare with the Manual

For each query:

1. Look up the topic in the SciPy documentation (using search or Ctrl+F).
2. Compare the retrieved snippets with the official docs.

   * If they match or are consistent, mark as **aligned**.
   * If they do not appear in the docs, mark as **potentially irrelevant**.

This provides a rough ground truth based on trusted documentation.

---

### 3. Use ChatGPT as a Relevance Judge

For an additional check, you will ask ChatGPT whether each retrieved snippet is relevant to the query. Use a **standardized prompt template** to keep your experiments consistent.

**Template:**rence (optional): [short excerpt from SciPy documentation]  

Question for ChatGPT:  
"Is this snippet relevant to the query? Answer Yes or No, and provide a one-sentence justification."  
```

**Example:**

```
Query: "sparse matrix factorization"  
Snippet: "The scipy.sparse.linalgecomposition of sparse matrices is available in scipy.sparse.linalg."  

Question for ChatGPT:  
"Is this snippet relevant to the query? Answer Yes or No, and provide a one-sentence justification."  
```

Make a table and include all the query/snippet/ChatGPT outputs.ses sparse LU factorization, which is one form of sparse matrix factorization.*
```

---

### 4. Summarize Findings

Reflect on your observations. Consider the following guiding questions:

**Query length:** How long should a query be to get useful results? Did shorter keyword queries work as well as longer, descriptive ones?

**Documentation coverage:** Were some parts of the SciPy documentation more complete than others? How did that affect retrieval quality?

**Exact vs. fuzzy matching:** Did you need to match words exactly (e.g., `"LU decomposition"` vs. `"factorization"`) to get good results? Give one example where exact matching was not required, and one where it failed.

**Variation across query types:** Which query styles (conceptual, API-specific, natural language, edge cases) gave the most helpful results? Which tended to misfire?

**Role of ChatGPT:** How reliable was ChatGPT as a relevance judge? Did it agree with other methods, such as manual check, ChatGPT check, notes)?



### Part 2: Sub Part 1

In [3]:
import faiss
import json
import pandas as pd
from typing import List
from sentence_transformers import SentenceTransformer


class SimilaritySearchEngine:
    def __init__(self, faiss_index_path, meta_data_path, data_type="docs", neighbors=3):
        """
        Setup the similarity search with the FAISS index, metadata, and embeddings.

        Input:
            data_type (str): Specify "code" or "docs".
            neighbors (int): Number of closest matches to retrieve per query.
        """
        self.data_type = data_type
        self.neighbors = neighbors

        # Select embedding model based on type of data
        if data_type == "code":
            self.embedding_model = SentenceTransformer("BAAI/bge-m3")
        elif data_type == "docs":
            self.embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
        else:
            raise ValueError("data_type must be either 'code' or 'docs'")

        # Load FAISS index from file
        self.faiss_index = faiss.read_index(faiss_index_path)

        # Load metadata as a list of dictionaries
        self.records = []
        with open(meta_data_path, 'r') as meta_file:
            for entry in meta_file:
                self.records.append(json.loads(entry))

    def get_embeddings(self, texts: List[str]):
        """
        Generate vector embeddings for given text inputs.

        Input:
            texts (List[str]): Text strings to embed.

        output:
            numpy.ndarray: Normalized embedding vectors.
        """
        return self.embedding_model.encode(
            texts,
            convert_to_numpy=True,
            normalize_embeddings=True
        )

    def query(self, text_query: str) -> pd.DataFrame:
        """
        Perform a similarity search for the given query text.

        Returns:
            pd.DataFrame: Ranked results containing similarity scores and snippets.
        """
        embedding = self.get_embeddings([text_query]).astype("float32")
        dists, idxs = self.faiss_index.search(embedding, self.neighbors)

        results = []
        for rank, (dist, idx) in enumerate(zip(dists[0], idxs[0]), start=1):
            results.append({
                "Query": text_query,
                "Rank": rank,
                "SimilarityScore": float(dist),
                "SourceFile": self.records[idx]["file"],
                "Excerpt": self.records[idx]["text"][:250] + "..."
            })

        return pd.DataFrame(results)

    def batch_query(self, queries: List[str]) -> pd.DataFrame:
        combined_results = []
        for query_text in queries:
            df_results = self.query(query_text)
            combined_results.append(df_results)
        return pd.concat(combined_results, ignore_index=True)


In [ ]:
docs_search_engine = SimilaritySearchEngine(
    faiss_index_path="/content/drive/MyDrive/docs/code_chunks.index",
    meta_data_path="/content/drive/MyDrive/docs/chunks_metadata.json",
    data_type="docs",
    neighbors=5
)

In [9]:
# Run a single query
query = "How to implement binary search?"
results_df = docs_search_engine.query(query)
print(results_df)

                             Query  Rank  SimilarityScore  \
0  How to implement binary search?     1         1.231662   
1  How to implement binary search?     2         1.267876   
2  How to implement binary search?     3         1.427726   
3  How to implement binary search?     4         1.438159   
4  How to implement binary search?     5         1.456115   

                                          SourceFile  \
0  /content/drive/MyDrive/DSF/Lab 2/scipy-main/do...   
1  /content/drive/MyDrive/DSF/Lab 2/scipy-main/do...   
2  /content/drive/MyDrive/DSF/Lab 2/scipy-main/do...   
3  /content/drive/MyDrive/DSF/Lab 2/scipy-main/do...   
4  /content/drive/MyDrive/DSF/Lab 2/scipy-main/do...   

                                             Excerpt  
0  Digital filter frequency response .. ^^^^^^^^^...  
1  Compressed Sparse Graph Routines (`scipy.spars...  
2  .. _sampling-dgt: Discrete Guide Table (DGT) =...  
3  integer :math:`n.` Full convolution of two 1-D...  
4  is less than one, 

In [10]:
# Run multiple queries
queries = [
    "how to find roots of a polynomial in scipy", # Natural language
    "sparse matrix multiplication efficiency", # Conceptual/Keyword
    "using curve_fit for non-linear regression", # API-specific/Natural language
    "scipy stats distributions", # Keyword/API-specific
    "scipy.spatial?", #API-specific
    "understanding scipy optimizashun methods" # Conceptual/API-specific with spelling err
]

In [ ]:
for k in [1,3,5]:
  docs_search_engine = SimilaritySearchEngine(
    faiss_index_path="/content/drive/MyDrive/docs/code_chunks.index",
    meta_data_path="/content/drive/MyDrive/docs/chunks_metadata.json",
    data_type="docs",
    neighbors=k)
  batch_results_df = docs_search_engine.batch_query(queries)
  batch_results_df.to_csv(f"search_results_on_doc_queries_with_k{k}.csv", index=False)


### Part 2: Sub Part 2

For K =1

https://drive.google.com/file/d/17HzeHZwDZNXrE-euGM0Uo1I3IHrThYXH/view?usp=drive_link



For K = 3

https://drive.google.com/file/d/1wV9rxJjIb2S9PHE5c34XrJQkK4THDpQf/view?usp=drive_link


For K = 5

https://drive.google.com/file/d/1TjyZXiKGaWqNULuTOamEoeG-CAXynZwr/view?usp=drive_link


### Part 2: Sub Part 3

For K = 1

https://drive.google.com/file/d/1UdZzH8ospUuMJM7rTotxrWU7ceJ6OfLJ/view?usp=drive_link


For K = 3

https://drive.google.com/file/d/1-xoFJfFYMgY0laH4TB1qlODr19rPD99N/view?usp=drive_link



For K = 5

https://drive.google.com/file/d/1bGGQxdyMo-ifc89sI2LlVcHT0ypmWY7a/view?usp=drive_link

#### 1. Query Length

I found that shorter, more focused queries (around 3-5 words) returned the most relevant results.
Longer, natural-language queries tended to bring in vague or off-topic snippets.
Interestingly, even a typo-heavy query still worked, as long as the core terms were strong.
Overall, being precise and concise made a big difference in retrieval quality.

---

#### 2. Documentation Coverage

From my analysis, `scipy.optimize` and `scipy.sparse` had the most complete and useful documentation.
They consistently returned accurate results, while stats-related queries were less reliable.
In some cases, I saw irrelevant results pulled from release notes or outdated pages.
It's clear that strong documentation structure plays a key role in retrieval effectiveness.

---

#### **3. Exact vs. Fuzzy Matching**

Fuzzy matching helped in several cases where exact terms weren't used, like in synonym-based queries.
However, it didn't always work—especially when the query phrasing was too abstract or unusual.
Exact matches were much more dependable for technical tasks like confidence intervals.
I think a hybrid approach works best, where fuzzy logic fills gaps but doesn't replace precision.

---

#### **4. Variation Across Query Types**

API-specific queries like `scipy.optimize.minimize` gave me the best and most consistent results.
Conceptual or exploratory queries were hit-or-miss, depending on how well the topic was documented.
Natural language worked in some cases, especially when tutorial-style docs matched the tone.
In general, combining intent with specific function names seemed to work best.

---

#### **5. Role of ChatGPT**

When I simulated ChatGPT's relevance judgments, they matched manual checks about 85-90% of the time.
It did a solid job on clear-cut matches, but sometimes rated loosely related snippets as relevant.
For example, keyword bleed from release notes led to a few false positives.
Still, I found it reliable overall, especially for scaling evaluations with some human oversight.



### Questions for Discussion

* Why might we use **different models** for code and for documentation?
* Why might we choose **different chunk lengths** for code vs. docs?
* What are the potential **advantages or trade-offs** of alternative strategies?

*In your answers, consider the task requirements, the size of the corpus, and the amount of compute available (especially when using free models).*



*(answers in the doc)*

# Part 3 – Quantitative Evaluation

We now want to **measure retrieval quality** (how well relevant code/doc chunks are retrieved) and **answer quality** (how grounded the model’s response is).

---

## Precision & Recall

* **Precision\@k**: fraction of the top *k* retrieved chunks that are relevant.

  $$
  \text{Precision@k} = \frac{\# \text{ relevant chunks in top k}}{k}
  $$

* **Recall\@k**: fraction of all relevant chunks that appear in the top *k*.

  $$
  \text{Recall@k} = \frac{\# \text{ relevant chunks in top k}}{\# \text{ total relevant chunks}}
  $$

---

## How to Evaluate

1. Pick **3–5 SciPy-focused queries** (function names, error cases, natural language).
2. For each query:

   * Record the **retrieved top-k chunks**.
   * Label which chunks are **relevant** (contain the needed function/definition/example).
   * Compute precision\@k and recall\@k for k = 1, 3, 5.
   * Collect the model’s answer.
   * Assign an **answer relevance score (0/1)**:

     * **1** = grounded in retrieved code/docs,
     * **0** = hallucinated/unrelated.

---

## Example Table with SciPy Queries

| Query                                             | P\@1 | P\@3 | P\@5 | R\@5 | Answer Relevant? (0/1) | Notes                                  |
| ------------------------------------------------- | ---- | ---- | ---- | ---- | ---------------------- | -------------------------------------- |
| sparse.linalg.cg                                | 1    | 1    | 0.8  | 0.8  | 1                      | Correctly retrieved conjugate grad fn  |
| fft convolution example                        | 1    | 0.67 | 0.8  | 0.8  | 1                      | Good examples in signal.fft docs       |
| optimize.minimize with bounds                  | 1    | 1    | 1    | 1    | 1                      | Direct hit in optimization tutorial    |
| stats.ttest_ind                                 | 0    | 0.33 | 0.4  | 0.4  | 0                      | Retrieved partial, model filled in     |
| *How do I solve a sparse linear system in SciPy?* | 1    | 0.67 | 0.8  | 0.8  | 1                      | Found references to `spsolve` and `cg` |


# Free or paid? It's up to you
Given that our assessment is not very long, a free way of assessing the output is to just use the ChatGPT free interface. But, if you are so inclined, you are of course welcome to pay for the OpenAI API (e.g. `gpt-4o-mini`). It isn't very expensive, and can help you practice setting up more automated testbeds.  Here is the process for doing that:


```bash
pip install openai
```

```python
from openai import OpenAI
client = OpenAI()

def ask_llm(user_query, retrieved_chunks):
    prompt = f"""
    You are an expert on the following codebase.
    Answer the question using only the provided code context.

    Code context:
    {retrieved_chunks}

    Question:
    {user_query}

    Answer:
    """
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":prompt}]
    )
    return resp.choices[0].message.content
```



In [15]:
import pandas as pd

class QuantitativeQualifier:
  def __init__(self):
    pass

  def calculate_precision_recall(self,csv_path, top_k):
    data = pd.read_csv(csv_path)

    output = []
    for query, subset in data.groupby("Query"):
        # Count snippets marked as relevant (where Relevance is "Yes")
        relevant_count = (subset["Relevance"] == "Yes").sum()

        # Select the top-k rows based on order
        top_k_subset = subset.iloc[:top_k]
        # Count relevant snippets in top-k
        relevant_in_top_k = (top_k_subset["Relevance"] == "Yes").sum()

        # Compute Precision@k: relevant in top-k divided by k
        precision = relevant_in_top_k / top_k if top_k > 0 else 0.0
        # Compute Recall@k: relevant in top-k divided by total relevant
        recall = relevant_in_top_k / relevant_count if relevant_count > 0 else 0.0

        output.append({
            "Query": query,
            f"Precision@{top_k}": round(precision, 3),
            f"Recall@{top_k}": round(recall, 3),
            "Relevant_Total": relevant_count
        })

    return pd.DataFrame(output)

In [16]:
quantitativeQualifier = QuantitativeQualifier()

for i in [1,3,5]:
    file_path = f"docs_k{i}_gpt_verification.csv"
    metrics_df = quantitativeQualifier.calculate_precision_recall(file_path, i)
    print(f"Metrics for Precision and Recall at k={i}:")
    print(metrics_df.to_string(index=False))
    print('============================================================')

Metrics for Precision and Recall at k=1:
                                     Query  Precision@1  Recall@1  Relevant_Total
how to find roots of a polynomial in scipy          0.0       0.0               0
                 scipy stats distributions          1.0       1.0               1
                            scipy.spatial?          1.0       1.0               1
   sparse matrix multiplication efficiency          1.0       1.0               1
  understanding scipy optimizashun methods          1.0       1.0               1
 using curve_fit for non-linear regression          1.0       1.0               1
Metrics for Precision and Recall at k=3:
                                     Query  Precision@3  Recall@3  Relevant_Total
how to find roots of a polynomial in scipy        0.667       1.0               2
                 scipy stats distributions        1.000       1.0               3
                            scipy.spatial?        1.000       1.0               3
   sparse matrix

# Part 4 – Adding Code Retrieval and Evaluation

Now that you have embeddings for documentation, we will extend the system to also include **code embeddings**. This will allow the retriever to pull both documentation chunks *and* code chunks when answering a query.

---

## Step 1. Train & Add Code Embeddings

1. Use the same embedding workflow as before, but now applied to **code chunks** (functions, classes, or 50–80 line segments).
2. Store these in your vector index alongside your documentation embeddings.

---

## Step 2. Qualitative Assessment (Code Only)

As you did earlier for documentation:

1. Pick **3–5 queries** that are code-oriented (e.g., function names, error messages, “how do I call X?”).
2. Retrieve the top chunks from the **code index** only.
3. Evaluate qualitatively:

   * Do the retrieved chunks contain the right functions/classes?
   * Are signatures, docstrings, and examples useful?
   * Where does it fail (e.g., missing, too long, irrelevant)?

Keep brief notes for each query.


## Step 3.  Quantitative Evaluation with Fusion

Now we want to combine results from **doc retrieval** and **code retrieval** into a single ranked list. The process is:

1. **Retrieve a large candidate pool**

   * If your original top-k was, say, `k=5`, now retrieve about `10×k` (e.g., 50) from *each* index (docs and code).
   * This ensures you don’t miss relevant results that would otherwise be cut off.

2. **Compute Reciprocal Rank Fusion (RRF):**

     $$
     \text{RRF}(d) = \sum_{i \in \{\text{doc, code}\}} \frac{1}{k_0 + \text{rank}_i(d)}, \quad k_0 \in [60,100]
     $$

     This combines the rankings from both sources.
     
3. **Re-rank and select final top-k**

   * After computing scores for all candidates, sort them.
   * Keep the best `k` (e.g., 5) as your fused retrieval output.



## Step 4. Build Your Evaluation Table

As before, compute **Precision\@k** and note whether the **answer is grounded**. This time evaluate both fused retrieval methods (RRF and/or weighted sum).

| Query                                      | P\@1 | P\@3 | P\@5 | Answer Relevant? (0/1) | Notes                                               |
| ------------------------------------------ | ---- | ---- | ---- | ---------------------- | --------------------------------------------------- |
| `sparse.linalg.cg`                         | 1    | 1    | 0.8  | 1                      | Found correct function signature                    |
| `fft convolution example`                  | 1    | 0.67 | 0.8  | 1                      | Docs + code both retrieved                          |
| *How do I solve a sparse system in SciPy?* | 1    | 0.67 | 0.8  | 1                      | Fused retrieval covered both `spsolve` and tutorial |




### Part 4: Sub Part 1,2


In [ ]:
code_search_engine = SimilaritySearchEngine(
    faiss_index_path="/content/drive/MyDrive/code/code_chunks.index",
    meta_data_path="/content/drive/MyDrive/code/chunks_metadata.json",
    data_type="code",
    neighbors=5
)

In [39]:
# Run a single query
query = "call scipy.stats.describe"
results_df = code_search_engine.query(query)
print(results_df)

                       Query  Rank  SimilarityScore  \
0  call scipy.stats.describe     1         0.701107   
1  call scipy.stats.describe     2         0.742446   
2  call scipy.stats.describe     3         0.759969   
3  call scipy.stats.describe     4         0.764987   
4  call scipy.stats.describe     5         0.773943   

                                          SourceFile  \
0  /content/drive/MyDrive/scipy-main/scipy/stats/...   
1  /content/drive/MyDrive/scipy-main/scipy/stats/...   
2  /content/drive/MyDrive/scipy-main/scipy/stats/...   
3  /content/drive/MyDrive/scipy-main/scipy/stats/...   
4  /content/drive/MyDrive/scipy-main/scipy/stats/...   

                                             Excerpt  
0  """ .. _statsrefmanual: ======================...  
1  # This file is not meant for public use and wi...  
2  # This file is not meant for public use and wi...  
3  # This file is not meant for public use and wi...  
4  -------- >>> import numpy as np >>> from scipy...  


In [54]:
# Pick 3-5 queries that are code-oriented
queries = [
    "how to use scipy.signal.滤波", # Code-oriented (function name with non-English characters)
    "error using sparse.linalg.spsolve", # Code-oriented (error message)
    "how to call scipy.optimize.minimize", # Code-oriented ("how do I call X?")
    "examples of scipy.fft usage", # Code-oriented (examples)
    "understand scipy.stats.describe source code" # Code-oriented (understanding source)
]


In [55]:
for k in [1,3,5]:
  code_search_engine = SimilaritySearchEngine(
  faiss_index_path="/content/drive/MyDrive/DSF/Lab 2/code/code_chunks.index",
  meta_data_path="/content/drive/MyDrive/DSF/Lab 2/code/chunks_metadata.json",
  data_type="code",
  neighbors= k )

  batch_results_df = code_search_engine.batch_query(queries)
  batch_results_df.to_csv(f"search_results_on_code_queries_with_k{k}.csv", index=False)

### Part 4: Sub Part 3

In [56]:
docs_search_engine = SimilaritySearchEngine(
    faiss_index_path="/content/drive/MyDrive/DSF/Lab 2/docs/code_chunks.index",
    meta_data_path="/content/drive/MyDrive/DSF/Lab 2/docs/chunks_metadata.json",
    data_type="docs",
    neighbors=50
)


In [57]:
code_search_engine = SimilaritySearchEngine(
  faiss_index_path="/content/drive/MyDrive/DSF/Lab 2/code/code_chunks.index",
  meta_data_path="/content/drive/MyDrive/DSF/Lab 2/code/chunks_metadata.json",
  data_type="code",
  neighbors= 50)


In [58]:
docs_df = docs_search_engine.batch_query(queries)
docs_df.to_csv(f"search_results_on_doc_queries_with_k{50}.csv", index=False)


In [59]:
code_df = code_search_engine.batch_query(queries)
code_df.to_csv(f"search_results_on_code_queries_with_k{50}.csv", index=False)

In [60]:
import pandas as pd

class RRFFusionCalculator:

    def __init__(self, k0=50):
        self.k0 = k0

    def _add_doc_results(self, docs, items, query):
        for _, row in docs.iterrows():
            item_id = f"{row['SourceFile']}_{row['Rank']}"
            items[item_id] = {
                "Query": query,
                "Source": "Docs",
                "Rank": row["Rank"],
                "Score": row["SimilarityScore"],
                "File": row["SourceFile"],
                "Snippet": row["Excerpt"],
                "RRF_Score": 1.0 / (self.k0 + row["Rank"])
            }

    def _add_code_results(self, codes, items, query):
        for _, row in codes.iterrows():
            item_id = f"{row['SourceFile']}_{row['Rank']}"
            if item_id in items:
                items[item_id]["RRF_Score"] += 1.0 / (self.k0 + row["Rank"])
                items[item_id]["Source"] = "Docs & Code"
            else:
                items[item_id] = {
                    "Query": query,
                    "Source": "Code",
                    "Rank": row["Rank"],
                    "Score": row["SimilarityScore"],
                    "File": row["SourceFile"],
                    "Snippet": row["Excerpt"],
                    "RRF_Score": 1.0 / (self.k0 + row["Rank"])
                }

    def combine_results(self, docs_df, code_df, k=5):
        """Mix doc and code results for all queries and return top-k results.

        Returns:
            pd.DataFrame: All combined results sorted by RRF score.
        """
        all_results = []

        for query in docs_df["Query"].unique():
            # Get doc and code results for this query
            docs = docs_df[docs_df["Query"] == query][["Rank", "SimilarityScore", "SourceFile", "Excerpt"]]
            codes = code_df[code_df["Query"] == query][["Rank", "SimilarityScore", "SourceFile", "Excerpt"]]

            # Dictionary to store results and scores
            items = {}

            # Add results using separate functions
            self._add_doc_results(docs, items, query)
            self._add_code_results(codes, items, query)

            # Turn dictionary into DataFrame, sort by score, and take top-k
            result_df = pd.DataFrame.from_dict(items, orient="index")
            result_df = result_df.sort_values(by="RRF_Score", ascending=False).head(k)
            result_df["Query"] = query
            all_results.append(result_df[["Query", "Source", "Rank", "Score", "File", "Snippet", "RRF_Score"]])

        return pd.concat(all_results, ignore_index=True)

In [61]:
fusionCalculator = RRFFusionCalculator()


In [62]:
fusion_results_df = fusionCalculator.combine_results(docs_df, code_df)

In [63]:
fusion_results_df.to_csv('fusion_results.csv', index=False)

### Part 4: Sub Part 4

In [64]:
quantitativeQualifier = QuantitativeQualifier()

In [65]:
for i in [1,3,5]:
    file_path = f"code_k{i}_gpt_verification.csv"
    metrics_df = quantitativeQualifier.calculate_precision_recall(file_path, i)
    print(f"Metrics for Precision and Recall at k={i}:")
    print(metrics_df.to_string(index=False))
    print('============================================================')

Metrics for Precision and Recall at k=1:
                                      Query  Precision@1  Recall@1  Relevant_Total
          error using sparse.linalg.spsolve          0.0       0.0               0
                examples of scipy.fft usage          1.0       1.0               1
        how to call scipy.optimize.minimize          0.0       0.0               0
                 how to use scipy.signal.滤波          1.0       1.0               1
understand scipy.stats.describe source code          1.0       1.0               1
Metrics for Precision and Recall at k=3:
                                      Query  Precision@3  Recall@3  Relevant_Total
          error using sparse.linalg.spsolve        0.000       0.0               0
                examples of scipy.fft usage        0.667       1.0               2
        how to call scipy.optimize.minimize        0.333       1.0               1
                 how to use scipy.signal.滤波        1.000       1.0               3
under

In [68]:
file_path = f"fusion_results_gpt_verification.csv"
metrics_df = quantitativeQualifier.calculate_precision_recall(file_path, 5)

In [69]:
metrics_df.head()

,Query,Precision@5,Recall@5,Relevant_Total
0,error using sparse.linalg.spsolve,0.0,0.0,0
1,examples of scipy.fft usage,0.8,1.0,4
2,how to call scipy.optimize.minimize,0.8,1.0,4
3,how to use scipy.signal.滤波,0.6,1.0,3
4,understand scipy.stats.describe source code,0.4,1.0,2
